In [ ]:
# just a few notes

* ellipse centered at $(x_0, y_0)$: $\frac{(x-x_0)^2}{a^2} + \frac{(y-y_0)^2}{b^2}$ = 1
 * parabola with lowest point $(x_0, y_0)$: $y = a(x - x_0)^2 + y_0$
    * another form, called standard form: $y = ax^2 + bx + c$
    * more about parabolas: https://www.khanacademy.org/math/precalculus/conics-precalc/focus-and-directrix-of-a-parabola/v/focus-and-directrix-introduction
* hyperbola: $\frac{x^2}{a^2} - \frac{y^2}{b^2} = 1$

In [ ]:
# w